## Chapter 9: Recommender
### Ex3: Musical Instruments
Dataset: Musical_Instruments_5.json
Read more about dataset: http://jmcauley.ucsd.edu/data/amazon/
(http://jmcauley.ucsd.edu/data/amazon/)
Requirement:
Read dataset
Pre-process data
Use "asin" (ProductID), "reviewerID" and overall (User's reviews for each product - rating) to
build model to predict overalls => Give recommendation for users.


In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
import findspark
findspark.init()

spark-2.4.0-bin-hadoop2.7/
spark-2.4.0-bin-hadoop2.7/python/
spark-2.4.0-bin-hadoop2.7/python/setup.cfg
spark-2.4.0-bin-hadoop2.7/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/resultiterable.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/shell.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/heapq3.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/join.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/version.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/rdd.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/java_gateway.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/find_spark_home.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/_globals.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/worker.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/accumulators.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/feature.py
spark-2.4.0-bin-hadoop2.7/python/pyspark

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
%cd '/content/gdrive/My Drive/LDS9_K273_ONLINE_Đinh Viết Trung'

/content/gdrive/My Drive/LDS9_K273_ONLINE_Đinh Viết Trung


In [4]:
# import libraries
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
from pyspark.sql.functions import mean, stddev, col, log
from pyspark.sql.functions import to_date, dayofweek, to_timestamp
from pyspark.sql import types 
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType
from pyspark.sql.functions import year, month
from pyspark.sql.functions import dayofmonth, weekofyear
from pyspark.sql.functions import split, explode
from pyspark.sql.functions import coalesce, first, lit
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import Bucketizer
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.functions import datediff
from pyspark.sql.functions import when

In [5]:
from pyspark import SparkContext
sc = SparkContext()

In [6]:
spark = SparkSession(sc)

In [7]:
data = spark.read.json("data/Musical_Instruments_5.json")
data.show(5,truncate=True)


+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|1384719342|  [0, 0]|    5.0|Not much to write...|02 28, 2014|A2IBPI20UZIR0U|cassandra tu "Yea...|                good|    1393545600|
|1384719342|[13, 14]|    5.0|The product does ...|03 16, 2013|A14VAT5EAX3D9S|                Jake|                Jake|    1363392000|
|1384719342|  [1, 1]|    5.0|The primary job o...|08 28, 2013|A195EZSQDW3E21|Rick Bennette "Ri...|It Does The Job Well|    1377648000|
|1384719342|  [0, 0]|    5.0|Nice windscreen p...|02 14, 2014|A2C00NNG1ZQQG2|RustyBill "Sunday...|GOOD WINDSCREEN F...|    1392336000|
|1384719342|  [0, 0]|    5.0|This pop filter i...|02 21

In [8]:
data_sub = data.select(['asin', 'overall', 'reviewerID'])
data_sub.count()

10261

In [9]:
from pyspark.sql.functions import col, udf
from pyspark.sql.functions import isnan, when, count, col
data_sub.show(5, truncate=True)


+----------+-------+--------------+
|      asin|overall|    reviewerID|
+----------+-------+--------------+
|1384719342|    5.0|A2IBPI20UZIR0U|
|1384719342|    5.0|A14VAT5EAX3D9S|
|1384719342|    5.0|A195EZSQDW3E21|
|1384719342|    5.0|A2C00NNG1ZQQG2|
|1384719342|    5.0| A94QU4C90B1AX|
+----------+-------+--------------+
only showing top 5 rows



In [10]:
data_sub.select([count(when(col(c).isNull(), c)).alias(c) for c in
 data_sub.columns]).toPandas().T

,0
asin,0
overall,0
reviewerID,0


In [11]:
# Distinct users and movies
users = data_sub.select("reviewerID").distinct().count()
products = data_sub.select("asin").distinct().count()
numerator = data_sub.count()
display(numerator, users, products)


10261

1429

900

In [12]:
# Number of ratings matrix could contain if no empty cells
denominator = users * products
denominator


1286100

In [13]:
#Calculating sparsity
sparsity = 1 - (numerator*1.0 / denominator)
print ("Sparsity: "), sparsity


Sparsity: 


(None, 0.992021615737501)

In [14]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
# Converting String to index
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col


In [15]:
# Create an indexer
indexer = StringIndexer(inputCol='asin', 
 outputCol='asin_idx')

# Indexer identifies categories in the data
indexer_model = indexer.fit(data_sub)

# Indexer creates a new column with numeric index values
data_indexed = indexer_model.transform(data_sub)

# Repeat the process for the other categorical feature
indexer1 = StringIndexer(inputCol='reviewerID', 
 outputCol='reviewerID_idx')
indexer1_model = indexer1.fit(data_indexed)
data_indexed = indexer1_model.transform(data_indexed)

In [16]:
data_indexed.show(5, truncate=True)

+----------+-------+--------------+--------+--------------+
|      asin|overall|    reviewerID|asin_idx|reviewerID_idx|
+----------+-------+--------------+--------+--------------+
|1384719342|    5.0|A2IBPI20UZIR0U|   781.0|          72.0|
|1384719342|    5.0|A14VAT5EAX3D9S|   781.0|         359.0|
|1384719342|    5.0|A195EZSQDW3E21|   781.0|         436.0|
|1384719342|    5.0|A2C00NNG1ZQQG2|   781.0|        1216.0|
|1384719342|    5.0| A94QU4C90B1AX|   781.0|        1137.0|
+----------+-------+--------------+--------+--------------+
only showing top 5 rows



In [17]:
data_indexed.select([count(when(col(c).isNull(), c)).alias(c) for c in
 data_indexed.columns]).toPandas().T

,0
asin,0
overall,0
reviewerID,0
asin_idx,0
reviewerID_idx,0


In [18]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data_indexed.randomSplit([0.8, 0.2])

In [19]:
# Creating ALS model and fitting data
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

als = ALS(maxIter=5, 
 regParam=0.09, 
 rank = 25,
 userCol="reviewerID_idx", 
 itemCol="asin_idx", 
 ratingCol="overall", 
 coldStartStrategy="drop",
 nonnegative=True)
model = als.fit(training)

In [20]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
predictions.select(["asin_idx", "reviewerID_idx", 
 "overall", "prediction"]).show(5)


+--------+--------------+-------+----------+
|asin_idx|reviewerID_idx|overall|prediction|
+--------+--------------+-------+----------+
|   148.0|          63.0|    4.0|  4.483797|
|   148.0|           0.0|    5.0|  4.234727|
|   463.0|         671.0|    5.0| 4.5373945|
|   463.0|         327.0|    5.0|  4.589314|
|   463.0|         408.0|    5.0| 4.0018005|
+--------+--------------+-------+----------+
only showing top 5 rows



In [21]:
evaluator = RegressionEvaluator(metricName="rmse", 
 labelCol="overall",
 predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))


Root-mean-square error = 1.153425977061427


In [22]:
# rmse nhỏ => tốt

In [23]:
# Providing Recommendations: for all users
# get 20 recommendations which have highest rating.
user_recs = model.recommendForAllUsers(20)


In [24]:
for user in user_recs.head(5):
    print(user)
    print("\n")


Row(reviewerID_idx=471, recommendations=[Row(asin_idx=771, rating=6.354895114898682), Row(asin_idx=359, rating=6.2927422523498535), Row(asin_idx=705, rating=6.244109630584717), Row(asin_idx=552, rating=6.177147388458252), Row(asin_idx=746, rating=6.130697727203369), Row(asin_idx=678, rating=6.12703275680542), Row(asin_idx=426, rating=6.104310989379883), Row(asin_idx=544, rating=6.065990924835205), Row(asin_idx=717, rating=6.0507025718688965), Row(asin_idx=886, rating=5.997840881347656), Row(asin_idx=234, rating=5.983365058898926), Row(asin_idx=754, rating=5.979045391082764), Row(asin_idx=853, rating=5.970083236694336), Row(asin_idx=328, rating=5.965728759765625), Row(asin_idx=765, rating=5.948724746704102), Row(asin_idx=622, rating=5.945672035217285), Row(asin_idx=157, rating=5.911734580993652), Row(asin_idx=895, rating=5.900039196014404), Row(asin_idx=514, rating=5.888000011444092), Row(asin_idx=478, rating=5.88259220123291)])


Row(reviewerID_idx=1342, recommendations=[Row(asin_idx=7

### Converting back to string form


In [25]:
import pandas as pd
recs=model.recommendForAllUsers(10).toPandas()
nrecs=recs.recommendations.apply(pd.Series) \
.merge(recs, right_index = True, left_index = True) \
.drop(["recommendations"], axis = 1) \
.melt(id_vars = ['reviewerID_idx'], value_name = "recommendation") \
.drop("variable", axis = 1) \
.dropna() 
nrecs=nrecs.sort_values('reviewerID_idx')
nrecs=pd.concat([nrecs['recommendation'].apply(pd.Series), 
nrecs['reviewerID_idx']], axis = 1)
nrecs.columns = [ 
    'ProductID_index',
    'Rating',
    'UserID_index' 
 ]


In [26]:
md=data_indexed.select(['reviewerID', 'reviewerID_idx', 
 'asin', 'asin_idx'])
md=md.toPandas()

dict1 =dict(zip(md['reviewerID_idx'],md['reviewerID']))
dict2=dict(zip(md['asin_idx'],md['asin']))

nrecs['reviewerID']=nrecs['UserID_index'].map(dict1)
nrecs['asin']=nrecs['ProductID_index'].map(dict2)
nrecs=nrecs.sort_values('reviewerID')

nrecs.reset_index(drop=True, inplace=True)
new=nrecs[['reviewerID','asin','Rating']]

new['recommendations'] = list(zip(new.asin, new.Rating))
res=new[['reviewerID','recommendations']] 
res_new=res['recommendations'].groupby([res.reviewerID])\
 .apply(list).reset_index()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [27]:
res_new

,reviewerID,recommendations
0,A00625243BI8W1SSZNLMD,"[(B0010CHS8E, 6.325232982635498), (B001E3BSKS,..."
1,A10044ECXDUVKS,"[(B000EEHN9S, 4.923856735229492), (B000RY68PA,..."
2,A102MU6ZC9H1N6,"[(B001C9R5P6, 5.965214252471924), (B001L8KE06,..."
3,A109JTUZXO61UY,"[(B0002CZVB4, 6.211723327636719), (B001C9R5P6,..."
4,A109ME7C09HM2M,"[(B0002CZVB4, 5.93190860748291), (B001E3BSKS, ..."
...,...,...
1424,AZJPNK73JF3XP,"[(B0007WPCKE, 5.614171028137207), (B000EEHN9S,..."
1425,AZMHABTPXVLG3,"[(B00BTGMI5O, 3.801311731338501), (B0002E1OVS,..."
1426,AZMIKIG4BB6BZ,"[(B0002D0B4K, 6.266716480255127), (B000U0DU34,..."
1427,AZPDO6FLSMLFP,"[(B000RY68PA, 5.604221820831299), (B0002D0B4K,..."
